In [ ]:
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import librosa
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
print(torch.cuda.is_available(),
torch.cuda.current_device(),
torch.cuda.device(0),
torch.cuda.get_device_name(0))
import librosa.display
from librosa.feature import melspectrogram
from IPython.display import Audio

In [ ]:
hop_length = 192
win_length = 6*hop_length

n_mels = 192
sample_rate = 16000
n_fft = 6*hop_length

spectrogram = T.Spectrogram(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
)

griffin_lim = T.GriffinLim(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
)

def plot_spectrogram(spec, title=None, ylabel='freq_bin', aspect='auto', xmax=None):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or 'Spectrogram (db)')
    axs.set_ylabel(ylabel)
    axs.set_xlabel('frame')
    im = axs.imshow(librosa.power_to_db(spec), origin='lower', aspect=aspect)
    if xmax:
        axs.set_xlim((0, xmax))
    fig.colorbar(im, ax=axs)
    plt.show(block=False)
    
def removearray(L,arr):
    ind = 0
    size = len(L)
    while ind != size and not np.array_equal(L[ind],arr):
        ind += 1
    if ind != size:
        L.pop(ind)
    else:
        raise ValueError('array not found in list.')

In [ ]:
waveform, sr = librosa.load(r"C:\Users\g_bab\Downloads\1 hour botw.mp3")

In [ ]:
print(type(waveform))
Audio(waveform, rate = 16000)

In [ ]:
list_samples = np.array_split(waveform, 3600)
print(type(list_samples), len(list_samples))
for sample in (list_samples):
    if len(sample) != 23560:
        print(len(sample))
        removearray(list_samples, sample)

In [ ]:
print(len(list_samples))

In [ ]:
spec = spectrogram(torch.from_numpy(list_samples[200]))
print(spec.size())
#print_stats(spec)
plot_spectrogram(spec, title='torchaudio')

In [ ]:
specs_list = []

In [ ]:
for sample in list_samples: 
    tensor = torch.from_numpy(sample)
    spec = spectrogram(tensor)
    
    specs_list.append(spec)
    #plot_spectrogram(spec[0], title='torchaudio')
print(type(specs_list[1]))

In [ ]:
#testing if the audio can be reconstructed well enough through Griffin-Lim
toaudio_list = []
for sample in specs_list[:100]:                #first 100 melspecs (np arrays) in a list
    sample = sample.detach().numpy()
    toaudio_list.append(sample)

In [ ]:
list_amp_values = []
for melspec in toaudio_list:
    audio_array = librosa.feature.inverse.mel_to_audio(melspec, sr = 16000)
    list_amp_values += list(audio_array)
playback = np.asarray(list_amp_values)


In [ ]:
Audio(playback, rate=44100)    